In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

In [ ]:
# make the plots bigger
plt.rcParams["figure.figsize"] = (10,10)

# Image segmentation 

Segmentation is the process of separating pieces out of the whole. In computer vision this tends to refer to separating groups of pixels and grouping them together.       
Segmentation is useful because it allows us to say "this is a ball" and "this is a square". There are many things that fall under segmentation in general, but here we will cover the one provided by OpenCV `findContours` function. 

Contours can be a bit difficult to explain because on face-value they look kind of like what edge detection algorithms do. However, they are indeed more powerful, because instead of an bitmap image, denoting a kind of a map of pixels that belong to an edge, contours provide us with the edge boundaries and their relationship to other boundaries.        
This makes a big difference because it allows us to test the properties of these objects. 

For now, let's quickly whip up an example of edges:

In [ ]:
from scipy import ndimage

img = np.zeros((256, 256), dtype=np.uint8)
img[64:-64, 64:-64] += 3
img[32:-32, 32:-32] += 2
img[16:-16, 16:-16] += 1
plt.imshow(img)

Let's repeat what we did in the previous excercise and detect edges:

In [ ]:
edges = cv2.Canny(img, 0, 1)
plt.imshow(edges)

# Contours

Everything you could possibly want to learn about contours can be found in the [OpenCV Contours Tutorials](https://docs.opencv.org/3.4/d3/d05/tutorial_py_table_of_contents_contours.html). At this point I would most heartily recommend that you at least cover the [Getting Started](https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html) portion of OpenCV tutorials because they will demistify what the function arguments are and what contours are.

After that feel free to jump back to this notebook where I will attempt to demistify what the function returns back. Unfortunately, due to the historical ties of OpenCV to C++ the returned values are somewhat confusing. More than some deep knowledge about contours, I aimed to impart in this notebook a sort-of how-to on code sleuthing on a set of practical examples. I think it's important because more than once you'll find yourself trying to wrap your head around a piece of OpenCV nonsense. We will eventually and periodically return to the topic at hand throughout the text, so there is a chance there is worthy information there, however, if this is not something you feel like you struggle with - jump straight to the second OpenCV tutorial on [Contour Features](https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html), [Contour Properties](https://docs.opencv.org/3.4/d1/d32/tutorial_py_contour_properties.html) and then jump to the bottom of the notebook titled "Fitting Minimal Area Rectangles". 

So, having either read the tutorial, or having pressed the "I believe" button and skipped it, let's move on:

In [ ]:
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

Let's visualize this in the same way the [Getting Started](https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html) tutorial showed us:

In [ ]:
cimg = np.zeros((256, 256), dtype=np.uint8)
cv2.drawContours(cimg, contours, -1, (255,255,255), 1)
plt.imshow(cimg)

Hmm, but that looks exactly the same as does the output of Canny edge detector?!             
So what even is the point of contours?

Let's look at what it actually returned to us:

In [ ]:
print("First Returned Value: contours")
print("    type: ", type(contours))
print("    len: ", len(contours))
print("    len(contours[0]): ", len(contours[0]))
print("First element of contours: ")
print(contours[0])

print()
print()
print()

print("Second Returned Value: hierarchy")
print("    type: ", type(hierarchy))
print("    len: ", len(hierarchy))
print("    len(hierarchy[0]): ", len(hierarchy[0]))
print("First element of hierarchy: ")
print(hierarchy[0])

So how do we make sense of this? Drawing them just makes an image of edges so that's a bust. How many contours did we even found? What are these points even? And how do we interpret the hierarchy matrix?

Here's what the documentation has to say about it:

```
    contours 	 Detected contours. Each contour is stored as a vector of points (e.g. std::vector<std::vector<cv::Point> >).
    hierarchy 	 Optional output vector (e.g. std::vector<cv::Vec4i>), containing information about the image topology. It has as many elements as the number of contours. For each i-th contour contours[i], the elements hierarchy[i][0] , hierarchy[i][1] , hierarchy[i][2] , and hierarchy[i][3] are set to 0-based indices in contours of the next and previous contours at the same hierarchical level, the first child contour and the parent contour, respectively. If for the contour i there are no next, previous, parent, or nested contours, the corresponding elements of hierarchy[i] will be negative.

Note
    In Python, hierarchy is nested inside a top level array. Use hierarchy[0][i] to access hierarchical elements of i-th contour. 
```

That doesn't really make sense to me to be honest, except that apparently the length of hierarchy is the number of detected contours. So let's try to manually plot some points:

In [ ]:
for point in contours[0]:
    plt.scatter(*point)

Let's decompose the above for loop. First let's make it clear what we want.                 

We want to plot all the point of a single contour.           
We want to use [matplotlib scatter](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html) function to do that.       
The `scatter` function expects the coordinates of a point in the plot, i.e. `plt.scatter(x, y)`, therefore we want to find a way to access the points of the first contour that give us the coordinates `(x, y)` of the edge of the contour. Seems reasonable enough. 

My personal natural instinct would be that `contours` is a list of detected contours. Each contour itself would be a list of points making up the edges of the contour. So accessing an elements of the list `contours` would given me n-th detected contour and accessing the elements of that would give me the `j-th` point of the `i-th` contour.  

So, my personal natural instinct would be to expect contour to be a list of points such as `[(x1, y1), (x2, y2), ...]`. Once I iterate over that list, I would iterate over individual points`(x1, y1)` then `(x2, y2)` and so on. If this were the case I could use the special operator `*`, called the unpacking operator, to plot my points on the graph.

The unpacking operator unpacks an array of values and passes them as individual positional arguments to a function. Effectively, when you want to call a function with multiple arguments, whose values you calculated and added to an array, can use the unpacking operator. Unpacking operator effectively performs the following: `*(x1, x2, x3...) --> x1, x2, x3...`, so when we plot `plt.scatter(*(x1, y1))` Python understands it as `plt.plot(x1, y1)`.                       
The unpacking operator has a big brother `**` that also works on dictionaries, in which case they would be passed in as keyword arguments instead of positional arguments. So having a dictionary of values like `d = {"arg1": 1, "arg2": 2}` and doing `function(**d)` would be interpreted by Python as `func(arg1=1, arg2=2)`.

So after that little detour back to the for-loop above. So far we expected that `contours` was a list of detected contours and that accessing an element of that list would give me the `i-th` detected contour and accessing an element of `i-th` contour gives me its `j-th` point as a tuple `(x_j, y_j)`.

 
It looks like we can select the i-th contour by `contours[i]`, so in our for loop `contours[0`] selects the first contour.   

However, if I try that then `plt.scatter` complains that it's missing a second coordinate.          
Trye printing what `point` is. If I print it I see output like `[(x1, y1)]` instead of `(x1, y1)`.        
It seems that, instead of what we expected to get, which is `(x1, y1)`, what we got is `[(x1, y1)]`.               

Therefore `contours` is a list of arrays of points, like `[ [(x1, y1)], [(x2, y2)], [(x3, y3)] ... ]` instead of `[(x1, y1), (x2, y2)...]`. 

So when we try to call `plt.scatter(*point)` it is interpreted as `plt.scatter((x1, y1))` instead of `plt.scatter(x1, y1)`. 

So, how do we fix this?

Why do we get a list of arrays of point-tuples and does that mean something? 

In short, no. The reason is that OpenCV is written `C++` and there is a little bit of conversion that occurs when transiting from `C++` types to `Python` types. Someone, long ago wrote something that made sense in C++ but didn't in Python. This is a famous OpenCV "feature".         
I guess nobody's perfect.

In any case, let's move on and inspect more contours. Let's plot all of the contours in separate plots with different titles and colors. After all there's only 6 of them.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10), sharex=True, sharey=True)
i = 0
for ax, cnt, color in zip(axes.ravel(), contours, ("red", "blue", "green", "orange","black", "gray")):
    for point in cnt:
        ax.scatter(*point[0], color=color)
        ax.set_title(f"Contour {i}")
    i+=1
    ax.set_xlim(0, 256)
    ax.set_ylim(0, 256)
plt.tight_layout()

Hmm, interesting! 

So it found the "same" contours as different contours, with the difference being only a couple of pixels. 

The contours are lines that track pixels of the same intensity - there is no requirement for this to be a closed curve whatsoever. It could be that the algorithm noticed the edges of the squares. where even we can see the pixel intensity is not as large, failed to close the contour and then double-backed on it. It could be that the fuzzy corners also made the contours detect the same one multiple times, just offset by a pixel or two. 

Whatever it is, we definitely need to be careful to properly filter our retrieved contours. So let's. 

Here, it would be of great benefit to check out all the kinds of properties and features contours have in the OpenCV  [Contour Features](https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html) and [Contour Properties](https://docs.opencv.org/3.4/d1/d32/tutorial_py_contour_properties.html) tutorials. They are really short and basically showcase why contours are so much more powerful than edge bitmaps - they are segmented!

If it didn't occur to you, we just plotted each individual contour - individually! This is a huge step forward compared to outputs of Canny edge detector. In combination with their properties we suddenly have significantly more information on the shapes, sizes, orientations and positions of the individual clumps of pixels in our image compare to just knowing which pixels might belong to a clump but not even knowing which one they belong to! 

To filter duplicated contours I looked up what property I could best take advantage of, decided it was area, and then removed contours of similar areas. 

In [ ]:
# this just makes life a bit easier
# do note that not all contours have the same number of points 
# and numpy hates this so we need to specify dtype=object to appease it
contours = np.array(contours, dtype=object)

areas = []
for cnt in contours:
    areas.append(cv2.contourArea(cnt))
print("All areas: ", areas)


# if the contours area are different by more than 5% of
# total image area we consider them different.
dimx, dimy = img.shape
total_image_area = dimx*dimy
area_threshold = 0.1 * total_image_area

# diff gives me the differences between neighbouring elements
# so 1st minus 2nd, 2nd minus 3rd etc...
# We can use this here because we know they are neatly ordered, 
# otherwise we might have to be a bit more clever.
diff = np.abs(np.diff(areas))
same_cnts = diff < area_threshold

# note that our indices will be 1 off because how diff works
# so we add the 1st element by hand and then add the rest after it
duplicates = [False]
duplicates.extend(same_cnts)
print("Duplicate contour: ", duplicates)

# so let's finally select the non-dupes
contours = contours[np.logical_not(duplicates)]

print(f"Selected {len(contours)} contours, rejected {duplicates.count(True)} contours as duplicates.")
cimg = np.zeros((256, 256), dtype=np.uint8)
cv2.drawContours(cimg, contours, -1, (255,255,255), 1)
plt.imshow(cimg)


So why have this section here and what have we learned?

Image algorithms are not perfect. They stumble, probably, more so than other algorithms from the CS domain. This is to be expected because Computer Vision is hard. Especially for computers. This same situation will occur again, soonest probably already in the following exercise where we use Hough Transform to detect lines, except in that case I won't provide the detailed walkthrough on how to debug issues, filter results and wrap your head around the weird OpenCV output formats. Hopefully, there are lessons and tricks you can take away from this hands on example, that you can apply there as well.

## Fitting minimal area rectangles

Let's take a different example, where we can see how poweful knowing moments (f.e. circumference, area, etc..) of our contours can be. Here we will experiment a little with a particular function that fits minimal area rectangles to contours, found in the [Contour Features](https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html) tutorial. 

To save us some time repeating the above, I've pre-made a bitmap, let's jump straight to detected edges.

In [ ]:
img = cv2.imread("images/bitmap.png", cv2.IMREAD_GRAYSCALE)

contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

cimg = np.zeros(img.shape, dtype=np.uint8)
cv2.drawContours(cimg, contours, -1, (255,255,255), 1)
plt.imshow(cimg)

The function of interest here, is `cv2.minAreaRect` function. Let's see what it's docstring can tell us

In [ ]:
cv2.minAreaRect?

Let's just fit all min area rectangles we can.

In [ ]:
cimg = np.zeros(img.shape, dtype=np.uint8)
treshold = 5

rectangles = []
for cnt in contours: 
    rect = cv2.minAreaRect(cnt) # [x, y, w, h, theta]
    rectangles.append(rect)
    
    # let's draw them simultaneously so we save some space
    box = cv2.boxPoints(rect)
    box = np.asarray(box, dtype=np.int32)
    cv2.fillPoly(cimg, [box], (255, 255, 255))

plt.imshow(cimg)

Hmm, interesting that `minAreaRect` found 2 equivalent solutions to fitting squares to circles, reminds you of something we just saw above perhaps?

If you feel like it, feel free to try to use some properties and features of contours to remove duplicate rectangles in the image, but as far as the main line of the notebook goes, we are interested only in the line features. How can we extract those?

Naturally, lines on images are really loooong looooooooong rectangles. So let's check their sides maybe?

In [ ]:
cimg = np.zeros(img.shape, dtype=np.uint8)

treshold = 5
lines = []
for cnt in contours: 
    rect = cv2.minAreaRect(cnt) # [x, y, w, h, theta]
    box = cv2.boxPoints(rect)
    box = np.asarray(box, dtype=np.int32)
    
    # lines are really long - i.e. one of their sides
    # is much larger than the other. Let's say if
    # one of the sides is threshold times the other one
    # we reject it!
    (x,y), (w, h), theta = rect
    if (w/h > treshold) or (h/w > treshold):
        lines.append(rect)
        cv2.fillPoly(cimg, [box], (255, 255, 255))
        
plt.imshow(cimg)    
print(lines)

So you see, contours do give us more information about our objects! What other properties or features of contours could be useful to us?

# Summary

* Image segmentation is the process of finding and grouping of pixels into objects and expressing their relations to one another. 
* Contour finding algorithms are a powerful tool for image segmentation
  * This is because they provide shape descriptions and ordering of the detected objects in our image
* Unfortunately they operate only on bitmaps (images with 1 or 0 only)
  * This requires us to either treshold our image beforehand, in such a way we produce edges, or run an edge detection algorithm
  * They can also be somewhat "dirty" - requiring us to filter out duplicates
* All lines on images can be considered as "rectangles" with, at least a minimum, side length of 1 pixel
  * Lines are long which means we can use the moments of our contours to find out whether or not the found contour looks like an 
    thin long object or if it's curcular or if etc.. 
  * We can also use these properties to figure out if they are duplicates or not.